In [1]:
import sys
sys.path.append('C:\Program Files\Anaconda3\Lib\site-packages')
#добавляю для себя, так у меня работают пакеты!

In [2]:
import pandas as pd
from tqdm import tqdm
import os
import re
from nltk import trigrams
from collections import Counter
import pymorphy2 # Лемматизация

## Скачиваем тексты wiki ##

In [20]:
import wikipedia

In [21]:
wikipedia.set_lang("ru")

In [28]:
random_pages_names = wikipedia.random(pages=500)

In [4]:
path_train_texts = 'C:/Users/Анна/YandexDisk/spell-check/wikiru/'

In [3]:
for random_name in tqdm(random_pages_names):
    page_text = []
    random_name = wikipedia.search(random_name)
    try:
        saved_page = wikipedia.page(random_name[0])
    except (wikipedia.exceptions.DisambiguationError,
            wikipedia.exceptions.PageError) as e:
        print("Error: {0}".format(e))
        continue
    try:
        page_text.append(saved_page.title)
        page_text.append(saved_page.summary)
        page_text.append(saved_page.content)
    except wikipedia.exceptions.DisambiguationError as e:
        print("Error: {0}".format(e))
        continue
             
   
    page_text = ' '.join(page_text)

    file_name = (path_train_texts+(saved_page.title) + '.txt')
    try:
        with open(file_name, 'w', encoding = 'utf-8') as fl:
            fl.write(page_text)
    except FileNotFoundError:
        continue

'for random_name in tqdm(random_pages_names):\n    page_text = []\n    random_name = wikipedia.search(random_name)\n    try:\n        saved_page = wikipedia.page(random_name[0])\n    except (wikipedia.exceptions.DisambiguationError,\n            wikipedia.exceptions.PageError) as e:\n        print("Error: {0}".format(e))\n        continue\n    try:\n        page_text.append(saved_page.title)\n        page_text.append(saved_page.summary)\n        page_text.append(saved_page.content)\n    except wikipedia.exceptions.DisambiguationError as e:\n        print("Error: {0}".format(e))\n        continue\n             \n   \n    page_text = \' \'.join(page_text)\n\n    file_name = (path_train_texts+(saved_page.title) + \'.txt\')\n    try:\n        with open(file_name, \'w\', encoding = \'utf-8\') as fl:\n            fl.write(page_text)\n    except FileNotFoundError:\n        continue'

In [6]:
def slurp(path):
    with open(path, 'r', encoding = 'utf-8') as file_object:
        return file_object.read()

## Препроцессинг ##
Создаем корпус текстов из Википедии.

In [4]:
def preprocess(texts):
    preprocessed_texts_list = []
    for text in texts:
        text = re.sub(r'[A-Za-z]', '', text)
        text = re.sub(r'[А-Я]{2,}', '', text)#убираем аббревиатуры
        lower_text = text.lower()#приводим к нижнему регистру, заменяем тире, чтобы потом оставить дефисы в словах
        #lower_text = re.sub(r'(\d)+-(\d)+','', lower_text)#удаление цифр типа 1999-2001
        del_new_line = re.sub(r'\n', '', lower_text)
        no_ref_text = re.sub(r'==литература(.)+|==источники(.)+', '', del_new_line)#удаляем все, что после рубрики "Сссылки"
        no_num_text = re.sub(r'\d', '', no_ref_text)#удаляем цифры
        extracted_text = re.findall(r'[а-яё]+', no_num_text)
        #разделяем на токены - последовательности букв с дефисами
        extracted_text = ' '.join(extracted_text)
        preprocessed_texts_list.append(extracted_text)
    return preprocessed_texts_list

In [4]:
train_texts_list = []
for root, dirs, files in os.walk(path_train_texts):
    for file_name in tqdm(files):
        input_path = path_train_texts + file_name
        text = slurp(input_path)
        train_texts_list.append(text)
preprocessed_train_texts = preprocess(train_texts_list)    

NameError: name 'path_train_texts' is not defined

In [81]:
preprocessed_train_texts[14]

'число одиннадцать натуральное число расположенное между числами и пятое простое число четвёртое число софи жермен простое число число также является числом софи жермен третье безопасное простое число простое число якобсталя одиннадцать натуральное число расположенное между числами и пятое простое число четвёртое число софи жермен простое число число также является числом софи жермен третье безопасное простое число простое число якобсталя свойства цифры первых пяти неотрицательных степеней числа являются элементами треугольника паскаля эндекахорон англ абстрактный правильный четырёхмерный политоп ячеек которого представляют собой полуикосаэдры число бога кубика рубика в метрике равно геометрия существует развёрток куба образующих подмножество гексамино и развёрток октаэдра образующих подмножество октиамондов из гексамино шестиклеточных полимино ровно являются чётными то есть при шахматной раскраске содержат чётное число светлых и чётное число тёмных квадратов дэвид а кларнер обнаружил 

In [82]:
train_wiki = pd.DataFrame(data = preprocessed_train_texts, columns=['wiki_texts'])

In [83]:
train_wiki.to_csv('train_wiki_2.csv', index = False, encoding = 'utf-8')

## Обучающий корпус -- тексты Википедии 
По нему мы будем считать априорные и условные вероятности слов и словарных биграм. 

In [4]:
train_wiki = pd.read_csv('/home/nst/mount/data/share/yd/comp_ling/spell-check/train_wiki_2.csv')

In [3]:
train_wiki = pd.read_csv('C:/Users/Анна/YandexDisk/spell-check/train_wiki_2.csv')

In [4]:
train_wiki.head()

,wiki_texts
0,долби долби лат астероид относящийся к группе ...
1,адрия адрия лат довольно крупный астероид из г...
2,гиппас гиппас лат др греч типичный троянский а...
3,гоголь гоголь лат типичный астероид главного п...
4,капрера капрера итал астероид главного пояса п...


## Обучающая выборка с ошибками

In [7]:
train = slurp('/home/nst/mount/data/share/yd/comp_ling/spell-check/training.txt')

In [10]:
train = slurp('C:/Users/Анна/YandexDisk/spell-check/training.txt')

In [8]:
train = train.split('\n\n')

In [9]:
train_df = pd.DataFrame(data = train, columns = ['train'])

In [10]:
train_df['wrong'], train_df['right'] = train_df['train'].str.split('\n',1).str

In [11]:
train_df.pop('train');

In [12]:
train_df.to_csv('train_df.csv', index = False, encoding="utf-8")

In [13]:
train_df = pd.read_csv('/home/nst/mount/data/linguistics_hse/comp_ling/spell_check/anyala_compling_repo/CompLing/spell_reading/train_df.csv')

In [7]:
train_df =  pd.read_csv('C:/Users/Анна/YandexDisk/spell-check/train_df.csv') ;

In [8]:
train_df.head()

,wrong,right
0,тоесть если дамой то только в том случае если ...,то есть если домой то только в том случае если...
1,но если у меня характер будет такойже как хотя...,но если у меня характер будет такой же как хот...
2,делать осьминогов из одуванчиков и куколок из ...,делать осьминогов из одуванчиков и куколок из ...
3,"Вальс в 5 па ( внимание, будет под другую музы...",Вальс в 5 па внимание будет под другую музыку ...
4,зима подходит к завершению а мне только сейчас...,зима подходит к завершению а мне только сейчас...


Уберем знаки препинания из обучающей выборки.

In [9]:
train_df.wrong = [' '.join(re.findall(r'[а-яё]+', text.lower())) for text in train_df.wrong]
train_df.right = [' '.join(re.findall(r'[а-яё]+', text.lower())) for text in train_df.right]

In [10]:
train_df.head()

,wrong,right
0,тоесть если дамой то только в том случае если ...,то есть если домой то только в том случае если...
1,но если у меня характер будет такойже как хотя...,но если у меня характер будет такой же как хот...
2,делать осьминогов из одуванчиков и куколок из ...,делать осьминогов из одуванчиков и куколок из ...
3,вальс в па внимание будет под другую музыку с ...,вальс в па внимание будет под другую музыку с ...
4,зима подходит к завершению а мне только сейчас...,зима подходит к завершению а мне только сейчас...


## Импорт словарей ##

Используем словарь RuCorpora (~ 380 000 токенов).

In [7]:
rucorpora_dict = slurp('/home/nst/mount/data/share/yd/comp_ling/spell-check/dict.opcorpora.txt')

FileNotFoundError: [Errno 2] No such file or directory: '/home/nst/mount/data/share/yd/comp_ling/spell-check/dict.opcorpora.txt'

In [8]:
rucorpora_dict = slurp('C:/Users/Анна/YandexDisk/spell-check/dict.opcorpora.txt')

In [9]:
rucorpora_dict = re.findall(r'[А-ЯЁ]+', rucorpora_dict)
rucorpora_dict = ' '.join(rucorpora_dict)
rucorpora_dict = rucorpora_dict.lower()
#rucorpora_dict = rucorpora_dict.split()

In [10]:
rucorpora_dict[:20]

'ёж ежа ежу ежа ежом '

## Строим символьные триграммы по тексту ##
Сначала мы хотели использовать триграмный фильтр для нахождения ошибок в тексте на основе редких триграм, но это оказалось не очень хорошей идеей, потому что было не очень ясно, какой порог встречаемости должен быть, чтобы отсекать редкие. В общем, модель плохо работала и пропускала много ошибок, мы решили от нее отказаться.

In [55]:
def get_trigrams(text): #строим триграммы по тексту
    trigrams_list = []
    for word in text.split():
        trigram = [''.join(char) for char in list(trigrams(word))]
        if trigram:
            trigrams_list.append(' '.join(trigram))
        else:
            trigrams_list.append(word)   

    return trigrams_list

In [56]:
print(train_df.wrong[0])
print(get_trigrams(train_df.wrong[0]))
print(' '.join(get_trigrams(train_df.wrong[0])).split())

тоесть если дамой то только в том случае если потеряю работу в сентябре
['тое оес ест сть', 'есл сли', 'дам амо мой', 'то', 'тол оль льк ько', 'в', 'том', 'слу луч уча чае', 'есл сли', 'пот оте тер еря ряю', 'раб або бот оту', 'в', 'сен ент нтя тяб ябр бре']
['тое', 'оес', 'ест', 'сть', 'есл', 'сли', 'дам', 'амо', 'мой', 'то', 'тол', 'оль', 'льк', 'ько', 'в', 'том', 'слу', 'луч', 'уча', 'чае', 'есл', 'сли', 'пот', 'оте', 'тер', 'еря', 'ряю', 'раб', 'або', 'бот', 'оту', 'в', 'сен', 'ент', 'нтя', 'тяб', 'ябр', 'бре']


##  Вероятности символьных триграмм в корпусе

In [57]:
def count_trigrams(corpus): #получаем вероятности триграмм в корпусе
    corpus_trigrams = []
    for text in corpus:
        tr = ' '.join(get_trigrams(text))
        corpus_trigrams.extend(tr.split())
    counts = dict(Counter(corpus_trigrams))
    for key, value in counts.items():
        counts[key] = value/len(corpus_trigrams)
    print(len(corpus_trigrams))    
    return counts

### Trigram wiki model

In [58]:
%%time
trigram_wiki_model = count_trigrams(train_wiki.wiki_texts)
#обучили 3-граммную модель на корпусе для получения вероятностей 3-грамм
#выводится общее число триграм в корпусе (c повторениями)

8297222
CPU times: user 17.1 s, sys: 472 ms, total: 17.5 s
Wall time: 17.8 s


In [59]:
trigram_wiki_model.get('есл') #вероятность 3-граммы "есл"

0.0001628255818634237

In [60]:
trigram_wiki_model.get('зрх') #такой 3-граммы нет, модель не возвращает ничего

In [34]:
len(trigram_wiki_model)

13679

In [35]:
10/8297222 #какой порог взять для проверки слова по словарю?

1.2052226636818925e-06

### Поиск ошибок в текстах 
На основе триграмного фильтра работал плохо. 

In [36]:
def search_errors(text): #поиск ошибок в тексте
    errors = []
    trigram_list = get_trigrams(text) #список строк, каждая строка - триграммы для одного слова
    #print(trigram_list)
    for i, trigrams_string in enumerate(trigram_list):
        for trigram in trigrams_string.split():
            #print(trigram_model.get(tr))
            if not(trigram_wiki_model.get(trigram)) or (trigram_wiki_model.get(trigram) < (10/8297222)):
                #print(trigram)
                #print(text.split()[i])
                #if morph.parse(text.split()[i])[0].normal_form not in rucorpora_dict:
                if text.split()[i] not in rucorpora_dict:
                    errors.append(text.split()[i])
    return list(set(errors))

In [37]:
search_errors(train_df.wrong[10])

['самыи', 'своеи']

In [38]:
wiki_errors = []
for text in tqdm(train_df.wrong):
    wiki_errors.extend(search_errors(text))

100%|██████████| 3000/3000 [00:50<00:00, 59.60it/s]


In [39]:
len(wiki_errors)

556

In [40]:
print(wiki_errors[:100])

['такойже', 'недюженные', 'пейзажы', 'самыи', 'своеи', 'щастье', 'никоглда', 'отличаються', 'расжился', 'подощрительно', 'преподше', 'замерзнуть', 'можнго', 'првязанность', 'любю', 'дабаа', 'йпри', 'поведйте', 'воопще', 'нмного', 'черезчур', 'неопределнное', 'обчным', 'всмысле', 'отношуся', 'мущщину', 'мачовсе', 'божеупаси', 'вопщем', 'болезньматки', 'ндфл', 'теща', 'лучшым', 'кагбэ', 'рыпки', 'каааак', 'тогдя', 'совршенно', 'нащщот', 'дефачка', 'оооочень', 'оплаьа', 'мужык', 'кефирчиком', 'обяснил', 'проблемнная', 'внутрениий', 'элкетроинструменты', 'всеже', 'хочеца', 'блетчли', 'такскаать', 'воопщим', 'разделеннный', 'расщифровать', 'прощаите', 'реклма', 'очеь', 'ооочень', 'хваостаюсь', 'маленбких', 'саиые', 'дежурцве', 'хотябы', 'наконецто', 'каааак', 'оооочень', 'щастье', 'собссно', 'продкты', 'удеается', 'бплгодаря', 'сногсшибальной', 'блекнет', 'посдравляю', 'ооочень', 'сеъли', 'подргрузился', 'всвязи', 'прошедшых', 'всмысле', 'вооот', 'обязаательно', 'погкуляем', 'удалитъ', 'мор

## Генерация кандидатов

Генерируем кандидатов находящихся на расстоянии одной операции друг от друга, а затем отсекаем лишних при помощи быстрого поиска по словарю.

### Быстрый поиск

In [11]:
from flashtext import KeywordProcessor
keyword_processor = KeywordProcessor()
keyword_processor.add_keywords_from_list(rucorpora_dict.split())

In [12]:
def get_all_candidates(word):
    letters = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
    cursor = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    delete = [L + R[1:] for (L,R) in cursor if R]
    insert = [L + l + R for l in letters for (L,R) in cursor]
    replace = [L + l + R[1:] for l in letters for (L,R) in cursor if R]
    transpose = [L + R[1] + R[0] + R[2:] for (L,R) in cursor if len(R)>1]
    return set(delete + insert + replace + transpose)

In [13]:
def kill_candidates(error_word):
    candidates = []
    for word in get_all_candidates(error_word):
        if keyword_processor.extract_keywords(word) == [word]:
            candidates.append(word)
    return candidates

In [14]:
print(len(get_all_candidates('пейзажы')))

495


In [15]:
%%time
print(kill_candidates('пейзажы'))

['пейзажны', 'пейзажа', 'пейзаж', 'пейзажи', 'пейзаже', 'пейзажу', 'пейзаны']
Wall time: 33 ms


## Вероятности на словарных биграммах nltk

In [16]:
import nltk

Для подсчета вероятностей и составления частотного словаря составляем список токенов по корпусу.

In [17]:
wiki_texts = train_wiki.wiki_texts
wiki_tokens = [word for text in wiki_texts for word in text.split()]
print(wiki_tokens[10:20])

['марса', 'он', 'был', 'открыт', 'декабря', 'года', 'американским', 'астрономом', 'любителем', 'чарльзом']


Получаем частотный словарь.

In [19]:
wiki_freq = nltk.FreqDist(wiki_tokens)

И словарь условных частот, который представляет собой словарь словарей для каждого взятого слова с указанием частоты совместной встречаемости.

In [20]:
wiki_cfreq = nltk.ConditionalFreqDist(nltk.bigrams(wiki_tokens))

Получаем словарь условных вероятностей на основе вики-корпуса при помощи которого мы будем определять вероятность кандидатов в правом и левом контексте и таким образом выбирать лучшего из списка.

In [21]:
cprob = nltk.ConditionalProbDist(wiki_cfreq, nltk.MLEProbDist)
print('p(русский язык) = %1.4f' %cprob['русский'].prob('язык'))
print('p(в доме) = %1.4f' %cprob['в'].prob('доме'))
print('p(солнечной системе) = %1.4f' %cprob['солнечной'].prob('системе'))

p(русский язык) = 0.1279
p(в доме) = 0.0011
p(солнечной системе) = 0.2558


Пишем функцию выбора лучшего кандидата.

In [22]:
def select_best_candidate(candidates, left_context, right_context):
    s = sum(dict(wiki_freq).values())    
#считаем условные вероятности в контексте
    if left_context: #если есть слово слева от ошибочного
        if right_context: #если есть слово справа
            cand_prob = [(cand, cprob[left_context].prob(cand) * 
                        cprob[cand].prob(right_context)) 
                         for cand in candidates]
        else:
            cand_prob = [(cand, cprob[left_context].prob(cand))
                         for cand in candidates]
            
    else:
        cand_prob = [(cand, cprob[cand].prob(right_context)) for cand in candidates]
#сортируем кандидатов по вероятности    
    cand_rate = sorted(cand_prob, key = lambda column:column[1], reverse = True) 
    if not(cand_rate[0][1]):
        cand_aprior = [(cand, dict(wiki_freq).get(cand)/s) 
                       if dict(wiki_freq).get(cand) else (cand, 0) 
                       for cand in candidates]
        cand_rate = sorted(cand_aprior, key = lambda column:column[1], reverse = True)
        
    return cand_rate[0][0], cand_rate[0][1]

In [50]:
select_best_candidate(['пейзаж', 'пейзажа', 'пейзажны', 'пейзаны', 'пейзаже', 'пейзажу', 'пейзажи'], '', 'за')

('пейзажи', 6.612932912897754e-06)

## Собираем спел-чекер

1. Поиск ошибок в тексте
2. Генерация кандидатов
3. Убийство кандидатов: если список слов оказывается пустым после проверки по словарю, возвращаем искомое слово (на случай, если его вдруг не оказалось в нашем словаре)
4. Считаем вероятность кандидата в контексте. Вставляем в текст кандидата с наибольшей вероятностью.
5. Возвращаем новый исправленный текст.

In [53]:
def spell_check(text):
    left_context = ''
    right_context = ''
    text_correct = ''
    probability = 0
    #поиск ошибок
    trigram_list = get_trigrams(text) #список строк, каждая строка - триграммы для одного слова
    for i, trigrams_string in enumerate(trigram_list):
        word = text.split()[i]
        for trigram in trigrams_string.split():
            if not(trigram_wiki_model.get(trigram)) or \
            (trigram_wiki_model.get(trigram) < (10/8297222)):
                if ' ' + word + ' ' not in rucorpora_dict:
                    #print(word)
                    
                    #генерируем кандидатов + убираем невозможные по 3-грамной модели, оставшиеся проверяем по словарю
                    candidates = kill_candidates(word)
                    
                    #print(candidates)
                    if (candidates): #если множество оставшихся кандидатов не пустое
                        if(i > 0): #если слово не первое в тексте, находим левый контекст
                            left_context = text.split()[i-1]
                        if i + 1 < len(text.split()): # если слово не последнее в тексте
                            right_context = text.split()[i+1]
                        #определяем наиболее вероятного
                        cand , probability = select_best_candidate(candidates, left_context, right_context)
                        #print('Prob:', probability)
                        if probability == 0:
                            word = select_with_whitespace(cand)
                        else:
                            word = cand
        text_correct += ' ' + word
    return text_correct.strip()
                

In [62]:
%%time
spell_check('какието пейзажы за окном незнакомые')

CPU times: user 560 ms, sys: 0 ns, total: 560 ms
Wall time: 609 ms


'какието пейзажи за окном незнакомые'

In [68]:
train_df.wrong[:15]

0     тоесть если дамой то только в том случае если ...
1     но если у меня характер будет такойже как хотя...
2     делать осьминогов из одуванчиков и куколок из ...
3     вальс в па внимание будет под другую музыку с ...
4     зима подходит к завершению а мне только сейчас...
5     ей в общем то все равно было на кого учиться л...
6     я пью кофе не ради проснуться а ради процесса ...
7     делая недюженные усилия над собой я таки выкар...
8     ударение в фамилие уже наверно плавно переехал...
9                  пейзажы за окном какие то незнакомые
10    самыи прикол в том что делая дела в своеи жизн...
11    все ничего но главная фишка этой игры как было...
12    тогдасоответственно говорят о язычной гортанно...
13     интересно как там работа объективов регулируется
14    да там их мало конешно и я бил силно пян и всп...
Name: wrong, dtype: object

Сначала мы попробовали собрать спел-чекер с испольpованием модели символьных триграм. 
В первых 15 предложениях исправились 2 ошибки:
недюжинные, пейзажи.

Остальные ошибки:
тоесть, такойже, тогдасоответственно, какието  -- не могли исправить
милион, фамилие, конешно даже не нашли. 

## Алгоритм без триграм

Проанализировав ошибки, мы решили собрать спел-чекер без триграмной модели, а также добавить генерацию кандидатов с пробелами, потому что раньше слова "такойже", "тоесть" никак не исправлялись.

In [23]:
def insert_whitespace(word):
    space = ' '
    cursor = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    insert = [L + l + R for l in space for (L,R) in cursor]
    return insert

In [24]:
def aprior_probability(word):
    word_aprior = 0
    s = sum(dict(wiki_freq).values())
    if dict(wiki_freq).get(word):
        word_aprior = dict(wiki_freq).get(word)/s
    else:
        word_aprior = 0
    return word_aprior

In [25]:
def select_with_whitespace(word):
    candidates = insert_whitespace(word)
    words = []
    probs = []
    for cand in candidates:
        cand = cand.split()
        if len(cand) == 2:
            probability = aprior_probability(cand[0])*aprior_probability(cand[1])
            words.append(' '.join(cand))
            probs.append(probability)
    cands = list(reversed(sorted(zip(words, probs), key=lambda x: x[1])))
   
    return cands[0]

In [26]:
cand, prob = select_with_whitespace('какието')
print(cand, prob)

какие то 2.3022487100499806e-08


In [27]:
def spell_check_fast_dict(text):
    left_context = ''
    right_context = ''
    text_correct = ''
    #probability_1 = 0
    #probability_2 = 0
    #cand_1 = ''
    
    #поиск ошибок
    text_list = text.split()
    for index, word in enumerate(text_list):
        if not keyword_processor.extract_keywords(word) == [word]:
            if not(aprior_probability(word)):
                cand_2, probability_2 = select_with_whitespace(word) #выбираем лучшего кандидата с пробелами

                candidates = kill_candidates(word) #генерируем кандидатов по edit distance + проверка по словарю               
                #print(candidates)
                if (candidates): #если множество оставшихся кандидатов не пустое
                    if(index > 0): #если слово не первое в тексте, находим левый контекст
                        left_context = text_list[index-1]
                    if index + 1 < len(text_list): # если слово не последнее в тексте
                        right_context = text_list[index+1]
                    cand_1, probability_1 = select_best_candidate(candidates, left_context, right_context)

                    if probability_2 > probability_1:
                        word = cand_2
                    else:
                        word = cand_1

                else:
                    if probability_2:
                        word = cand_2
                                
        text_correct += ' ' + word
    return text_correct.strip()
                

In [74]:
print(train_df.wrong.values[:2])

['тоесть если дамой то только в том случае если потеряю работу в сентябре'
 'но если у меня характер будет такойже как хотя бы у одного из них я продам свою почку а на все вырученные деньги накуплю герлянд']


In [81]:
%%time
spell_check_fast_dict('герлянд много не бывает такойже')

CPU times: user 2.68 s, sys: 4 ms, total: 2.68 s
Wall time: 2.79 s


'гирлянд много не бывает такой же'

In [77]:
print(train_df.wrong[6])

я пью кофе не ради проснуться а ради процесса сделать себе вкусный напиток в красивую чашку положить серебрянной ложкой сахар добавить горячего молока чтоб кофе оставался еще дольше горячим и дымчатым и наслаждаться кайфовать медитировать в течении хотя бы нескольких минут в день


In [83]:
%%time
spell_check_fast_dict(train_df.wrong[0])

CPU times: user 1.4 s, sys: 8 ms, total: 1.41 s
Wall time: 1.51 s


'тесть если дамой то только в том случае если потеряю работу в сентябре'

In [79]:
x = train_df.wrong[:500]

In [ ]:
%%time
corrected = [spell_check_fast_dict(text) for text in tqdm(x)]

In [154]:
train_small = train_df[:500]
train_small['corrected'] = corrected

/home/nst/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [156]:
train_small.head()

,wrong,right,corrected
0,тоесть если дамой то только в том случае если ...,то есть если домой то только в том случае если...,тесть если дамой то только в том случае если п...
1,но если у меня характер будет такойже как хотя...,но если у меня характер будет такой же как хот...,но если у меня характер будет такой же как хот...
2,делать осьминогов из одуванчиков и куколок из ...,делать осьминогов из одуванчиков и куколок из ...,делать осьминогов из одуванчиков и куколок из ...
3,вальс в па внимание будет под другую музыку с ...,вальс в па внимание будет под другую музыку с ...,вальс в па внимание будет под другую музыку с ...
4,зима подходит к завершению а мне только сейчас...,зима подходит к завершению а мне только сейчас...,зима подходит к завершению а мне только сейчас...


In [160]:
errors = train_small[train_small.right != train_small.corrected]

In [163]:
errors

,wrong,right,corrected
0,тоесть если дамой то только в том случае если ...,то есть если домой то только в том случае если...,тесть если дамой то только в том случае если п...
6,я пью кофе не ради проснуться а ради процесса ...,я пью кофе не ради проснуться а ради процесса ...,я пью кофе не ради проснуться а ради процесса ...
7,делая недюженные усилия над собой я таки выкар...,делая недюжинные усилия над собой я таки выкар...,делая недюжинные усилия над собой я таки выкар...
8,ударение в фамилие уже наверно плавно переехал...,ударение в фамилии уже наверно плавно переехал...,ударение в фамилия уже наверно плавно переехал...
10,самыи прикол в том что делая дела в своеи жизн...,самый прикол в том что делая дела в своей жизн...,самых прикол в том что делая дела в своей жизн...
14,да там их мало конешно и я бил силно пян и всп...,да там их мало конечно и я был сильно пьян и в...,да там их мало конечно и я бил сильно ян и всп...
15,на черном фоне белый квадрат а в нем написанно...,на черном фоне белый квадрат а в нем написано ...,на чёрном фоне белый квадрат а в нем написано ...
18,парочка прошлогодних снимков с правнуком племе...,парочка прошлогодних снимков с правнуком племя...,парочка прошлогодних снимков с правнуком племе...
19,музончик конечно зачетный и почемуто до боли з...,музончик конечно зачетный и почему то до боли ...,музончик конечно заметный и заметный до боли з...
20,в связи с заявлениями тимошенко об ухудшении с...,в связи с заявлениями тимошенко об ухудшении с...,в связи с заявлениями тимошенко об ухудшении с...


### Testing

In [32]:
test = slurp('./test (1).txt')

In [33]:
test = test.split('\n\n')

In [35]:
test_df = pd.DataFrame(data = test, columns = ['test'])

In [36]:
test_df['wrong'], test_df['right'] = test_df['test'].str.split('\n',1).str

In [37]:
test_df.pop('test');

In [38]:
test_df.wrong = [' '.join(re.findall(r'[а-яё]+', text.lower())) for text in test_df.wrong]
test_df.right = [' '.join(re.findall(r'[а-яё]+', text.lower())) for text in test_df.right]

In [39]:
test_df.head()

,wrong,right
0,ну вот собственно и день весь вышел,ну вот собственно и день весь вышел
1,корридоры казались огромными было довольно пус...,коридоры казались огромными было довольно пуст...
2,а так все впринципе нормально,а так все в принципе нормально
3,надеюсь порисовать но посмотрю какая будеть по...,надеюсь порисовать но посмотрю какая будет погода
4,хотела посчитать скока я за седня выпила воды,хотела посчитать сколько я за сегодня выпила воды


In [40]:
len(test_df)

1008

In [41]:
corrected = [spell_check_fast_dict(text) for text in tqdm(test_df.wrong)]

100%|██████████████████████████████████████| 1008/1008 [37:18<00:00,  2.22s/it]


In [42]:
with open('answers.txt', 'w', encoding = 'utf-8') as f:
    f.write('\n'.join(corrected))
    

In [43]:
source = test_df.wrong
correct = test_df.right

In [44]:
with open('source.txt', 'w', encoding = 'utf-8') as f:
    f.write('\n'.join(source))

In [45]:
with open('correct.txt', 'w', encoding = 'utf-8') as f:
    f.write('\n'.join(correct))